### Import modules and constants

In [1]:
import json
import os
import argparse
from collections import Counter
import csv

baseline_packets = {"registered": {"Monday": 11709971, "Tuesday": 11551954, "Wednesday": 13788878, "Thursday": 9322025, "Friday": 9997874}}

packets_to_clone=200
countmin_time_threshold=10
countmin_width=16384

### Define the functions that can read the alerts and number of packets redirected from the results

In [2]:
def read_snort_alerts(alert_filepath):
	no_duplicates_data = {}
	data = []
	counter = Counter()
	file = open(alert_filepath)
	if os.path.getsize(alert_filepath) > 0:
		try:
			for line in file.readlines():
				parsed_line = json.loads(line)
				data.append(parsed_line)

				entry_key = str(parsed_line["pkt_num"]) + parsed_line["rule"] + parsed_line["timestamp"]
				if entry_key not in no_duplicates_data:
					no_duplicates_data[entry_key] = line
				
				counter[parsed_line["rule"]]+=1
		except Exception as e:
			print("JSON error: ", e)
	file.close()
	return data, counter, no_duplicates_data

def read_amt_of_cloned_pkts(p4_output_filepath):
	try:
		with open(p4_output_filepath) as file:
			packets_cloned = 0
			get_data = False
			previous_line = ""
			for line in file:
				if "Counter name:  MyEgress.cloned_to_ids" in previous_line and "Index (port):  index: 2" in line:
					get_data = True
				
				if "packet_count" in line and get_data==1:
					packets_cloned += int(line.split()[1])
					get_data = False

				previous_line = line
	except:
		print("No output file in this folder")

	return packets_cloned

### Read baseline alerts

In [3]:
baseline_alerts_folder="../../baseline_alerts/CICIDS2017/alerts_snort3-registered"

def read_baseline(baseline_folder):
	baseline_data = {}
	for alert_file in os.listdir(baseline_folder):
		item_fullpath = os.path.join(baseline_folder, alert_file)

		raw_data, rules_counter, no_duplicates_data =  read_snort_alerts(item_fullpath)
		baseline_data[alert_file.split(".")[0]]={"alerts": no_duplicates_data, "counter": rules_counter}
	
	return baseline_data

baseline_data = read_baseline(baseline_alerts_folder)

### Read the results of one complete experiment: Monday, Tuesday, etc.

In [4]:
def read_experiments_data(experiments_data_folder):
	folder_data = {}
	for item in os.listdir(experiments_data_folder):
		item_fullpath = os.path.join(experiments_data_folder, item)
		if os.path.isfile(item_fullpath):
			continue

		no_duplicates_data = {}
		raw_data = []
		rules_counter = Counter()
		for subdir in os.listdir(item_fullpath):
			if os.path.isfile(os.path.join(item_fullpath, subdir)):
				continue

			alert_file = os.path.join(item_fullpath, subdir) + "/alert_json.txt"
			data, counter, no_duplicates =  read_snort_alerts(alert_file)
			raw_data.extend(data)
			rules_counter.update(counter)
			for key, value in no_duplicates.items():
				if key not in no_duplicates_data:
					no_duplicates_data[key] = value

		packets_cloned = read_amt_of_cloned_pkts(os.path.join(item_fullpath, "output.txt"))
		folder_data[item]={"alerts": no_duplicates_data, "counter": rules_counter, "packets_cloned": packets_cloned}
	return folder_data

### Parse the parameters evaluation experiments

In [5]:
input_folder = "../../../experiments/final_evaluation_results"
output_folder = "./"
csv_data = []
for item in os.listdir(input_folder):
    item_fullpath = os.path.join(input_folder, item)
    if os.path.isfile(item_fullpath):
        continue

    folder_name_elements = item.split("_") # linear_simple_100_registered
    experiments_data = read_experiments_data(item_fullpath)

    for key, data in experiments_data.items():
        csv_line = {}

        
        csv_line["PCAP"] = key
        csv_line["Topology"] = folder_name_elements[0]
        csv_line["Offloading algorithm"] = folder_name_elements[1]
        csv_line["Available space (%)"] = folder_name_elements[2]
        csv_line["Alerts"] = len(data["alerts"])
        csv_line["% of alerts relative to the baseline"] =  len(data["alerts"])/len(baseline_data[key]["alerts"])
        csv_line["Packets cloned to NIDS"] = data["packets_cloned"]
        csv_line["% of packets cloned to NIDS relative to the baseline"] =  \
                 data["packets_cloned"]/baseline_packets[folder_name_elements[3]][key]
        csv_data.append(csv_line)

keys = csv_data[0].keys()
output_file = output_folder + "/final_evaluation.csv"
with open(output_file, 'w') as file:
    w = csv.DictWriter(file, keys)
    w.writeheader()

    for line in csv_data:
        w.writerow(line)

### Importing data mining and plotting libraries

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


final_eval_df = pd.read_csv(output_file, sep=",")
final_eval_df

,PCAP,Topology,Offloading algorithm,Available space (%),Alerts,% of alerts relative to the baseline,Packets cloned to NIDS,% of packets cloned to NIDS relative to the baseline
0,Tuesday,linear,simple,50,52727,0.628151,2375455,0.205632
1,Thursday,linear,simple,50,55274,0.744180,2164234,0.232164
2,Wednesday,linear,simple,50,57581,0.138718,3823853,0.277314
3,Monday,linear,simple,50,60215,0.913691,2382962,0.203499
4,Friday,linear,simple,50,62823,0.929747,2937751,0.293838
5,Tuesday,linear,simple,100,52662,0.627377,2461318,0.213065
6,Thursday,linear,simple,100,54864,0.738660,2244248,0.240747
7,Wednesday,linear,simple,100,57419,0.138328,3942710,0.285934
8,Monday,linear,simple,100,60280,0.914678,2502690,0.213723
9,Friday,linear,simple,100,62585,0.926225,3085602,0.308626


In [13]:
t_10_df = final_eval_df[(final_eval_df["Topology"]=="linear") & (final_eval_df["PCAP"]=="Wednesday")]
t_10_df

,PCAP,Topology,Offloading algorithm,Available space (%),Alerts,% of alerts relative to the baseline,Packets cloned to NIDS,% of packets cloned to NIDS relative to the baseline
2,Wednesday,linear,simple,50,57581,0.138718,3823853,0.277314
7,Wednesday,linear,simple,100,57419,0.138328,3942710,0.285934
12,Wednesday,linear,simple,75,57710,0.139029,3943301,0.285977
